# Implementação de algoritmo para usar as classes de simulação de array de microfones e GSC para o SESA Dataset

A ideia desse jupyter é criar um algoritmo que, dadas a pasta de origem e as de destino, simule um array de microfones com os áudios da pasta de origem e salve os sinais sem beamforming, com beamforming e com GSC nas pastas de destino.

In [1]:
import os
import librosa
import math
import shutil
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio

#### Importando as classes criadas

In [2]:
import sys
sys.path.append('/home/dimi/Programming/IC2019/DOA/Python')

from GeneralizedSidelobeCancellerClass import GeneralizedSidelobeCanceller
from SimularArrayMicrofonesClass import SimularArrayMicrofones

## Definindo uma boa amplitude para o ruído

Pela definição, se a média do sinal for 0, pode-se aproximar a energia do sinal pela variância. A SNR pode ser definida como 10 * log_10 (var(sinal)/var(ruído)).

"Engineers consider a SNR of 2 (3 dB) to be the boundary between low and high SNRs".

http://www.scholarpedia.org/article/Signal-to-noise_ratio

In [3]:
def calcularSNR(sinal, ruido):
    return 10 * math.log((np.var(sinal)/np.var(ruido)), 10)

In [4]:
def calcularEnerigaRuidoDadoSNR(snrDesejada, energiaSinal):
    return energiaSinal/(10**(snrDesejada/10))

A função abaixo é exatamente a mesma que está sendo usada na classe SimularArrayMicrofones. Essa parte do jupyter vai servir apenas para achar uma boa SNR para realizar os testes.

In [ ]:
def gerarRuidoBranco(qtdAmostras, energiaRuido):

    media        = 0
    desvioPadrao = 1

    return np.random.normal(media, desvioPadrao, size=qtdAmostras) * energiaRuido**(1/2)

O sinal que será aberto para fazer o teste **JÁ DEVE ESTAR NORMALIZADO PELA ENERGIA**, assim, fica muito mais fácil calcular a SNR.

In [ ]:
caminho = "/home/dimi/Downloads/Datasets/SESA/SESA_Normalizado/test/casual_000.wav"
sinalOriginal, freqAmostragem = librosa.load(caminho, sr=None, mono=True)

snrDesejada  = 20
energiaRuido = calcularEnerigaRuidoDadoSNR(snrDesejada, 1) 

ruido = gerarRuidoBranco(len(sinalOriginal), energiaRuido)

plt.plot(sinalOriginal, label="Sinal")
plt.plot(ruido, label="Ruído")
plt.legend()
plt.show()

In [ ]:
print("Energia do sinal:", np.var(sinalOriginal))
print("Energia do ruído:", energiaRuido)
print("SNR:\t\t ", calcularSNR(sinalOriginal, ruido))

In [ ]:
print("Original")
Audio(data=sinalOriginal, rate=freqAmostragem)

In [ ]:
print("Com ruído")
sinalComRuido = sinalOriginal + ruido
Audio(data=sinalComRuido, rate=freqAmostragem)

## Rodando o algoritmo para várias SNRs diferentes na pasta de teste do SESA

In [5]:
# DEFININDO PARAMETROS INICIAIS
pastaOrigem  = '/home/dimi/Downloads/Datasets/SESA/SESA_Normalizado/test/'
pastaDestino = '/home/dimi/Downloads/Datasets/SESA/SESA_Normalizado/outros/'

# LISTANDO OS ARQUIVOS NO DIRETORIO DE ORIGEM
arrayArquivos = os.listdir(pastaOrigem)
totalArquivos = len(arrayArquivos)

# CRIANDO UM ARRAY PARA GUARDAR O TEMPO MEDIO PARA PROCESSAR CADA ARQUIVO PARA CADA SNR DIFERENTE
arrayTempoGSCCadaSNR = []

# PARA VARIOS SNRs
for snrAtual in range(-20, 51):
    
    print("\nINICIANDO A SIMULAÇÃO COM SNR", snrAtual)
    
    # TENHO QUE CRIAR A PASTA PARA ESSA SNR
    pastaDestinoSNRAtual = pastaDestino + "SNR_" + str(snrAtual) + "dB/"
    
    # SE ELA JA EXISTIR EU EXCLUO TUDO
    if os.path.isdir(pastaDestinoSNRAtual) == True:
        shutil.rmtree(pastaDestinoSNRAtual)
        print("Já havia uma pasta para os arquivos com essa SNR e portanto ela foi excluída")
        
    # AGORA SIM EU CRIO AS PASTAS E SUBPASTAS
    os.mkdir(pastaDestinoSNRAtual)
    os.mkdir(pastaDestinoSNRAtual + "testeSemBeamforming/")
    os.mkdir(pastaDestinoSNRAtual + "testeBeamforming/")
    os.mkdir(pastaDestinoSNRAtual + "testeGSC/")
    
    # PRECISO CALCULAR QUAL VAI SER A ENERGIA DO RUIDO PARA ESSA SNR
    energiaRuido = calcularEnerigaRuidoDadoSNR(snrAtual, 1)
    
    # ARRAY PARA GUARDAR O TEMPO PARA PROCESSAR O GSC DE CADA ARQUIVO
    arrayTempoGSCCadaArquivoSNRAtual = []
        
    # PARA CADA ARQUIVO NA PASTA DE ORIGEM
    for i, arquivoAtual in enumerate(arrayArquivos):

        print("\nArquivo " + str(i+1) + " de " + str(totalArquivos) + " -> " + str(100*((i+1)/totalArquivos)) + "%")

        # EU SIMULO UM ARRAY DE MICROFONES
        caminhoArquivo = pastaOrigem + arquivoAtual
        objSimularArrayMics = SimularArrayMicrofones(caminhoArquivo, energiaRuido=energiaRuido)
        arraySinaisSimulados, arrayDelays, freqAmostragem = objSimularArrayMics.obterResultado()

        # APLICO O BEAMFORMING E GSC (SEM QUE A CLASSE SAIBA QUAIS SAO OS DELAYS)
        objGSC = GeneralizedSidelobeCanceller(arraySinaisSimulados)
        sinalSemBeamforming, sinalBeamformado, sinalFinalGSC = objGSC.obterSinaisResultantes()
        arrayTempoGSCCadaArquivoSNRAtual.append(objGSC.tempoProcessamentoGSC)

        # E POR FIM SALVO NAS PASTAS DE DESTINO
        librosa.output.write_wav(pastaDestinoSNRAtual + "testeSemBeamforming/" + arquivoAtual, sinalSemBeamforming, freqAmostragem)
        librosa.output.write_wav(pastaDestinoSNRAtual + "testeBeamforming/" + arquivoAtual, sinalBeamformado, freqAmostragem)
        librosa.output.write_wav(pastaDestinoSNRAtual + "testeGSC/" + arquivoAtual, sinalFinalGSC, freqAmostragem)
    
    # DEPOIS DE PROCESSAR TODOS OS ARQUIVOS PARA A SNR ATUAL, EU COLOCO O TEMPO DE PROCESSAMENTO MEDIO NO ARRAY DE TEMPOS DE CADA SNR
    mediaTempoGSCSNRAtual = np.mean(arrayTempoGSCCadaArquivoSNRAtual)
    arrayTempoGSCCadaSNR.append([snrAtual, mediaTempoGSCSNRAtual])
    print("\nTempo médio para processar o GSC em cada arquivo com SNR " + str(snrAtual) + ": " + str(mediaTempoGSCSNRAtual))
    
# DEPOIS QUE ACABAR TUDO, EU PRECISO SALVAR AS INFORMACOES DE TEMPO DE PROCESSAMENTO
dataframe = pd.DataFrame(data=arrayTempoGSCCadaSNR, columns=["SNR", "Tempo médio para processar o GSC em cada arquivo"])
dataframe.to_csv(pastaDestino + "tempoProcessamentoGSC" + str(time.time()) + ".csv", index=False)


INICIANDO A SIMULAÇÃO COM SNR -20

Arquivo 1 de 105 -> 0.9523809523809524%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.659318208694458

Arquivo 2 de 105 -> 1.9047619047619049%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.073137044906616

Arquivo 3 de 105 -> 2.857142857142857%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.9462647438049316

Arquivo 4 de 105 -> 3.8095238095238098%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.278667688369751

Arquivo 5 de 105 -> 4.761904761904762%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.5091812610626221

Arquivo 6 de 105 -> 5.714285714285714%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.7367432117462158

Arqu

Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.9764716625213623

Arquivo 53 de 105 -> 50.476190476190474%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.7845070362091064

Arquivo 54 de 105 -> 51.42857142857142%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.8841896057128906

Arquivo 55 de 105 -> 52.38095238095239%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.4416258335113525

Arquivo 56 de 105 -> 53.333333333333336%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.5766720771789551

Arquivo 57 de 105 -> 54.285714285714285%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.6014516353607178

Arquivo 58 de 105 -> 55.23809523809524%
Iniciando a filtragem adaptativ

Tempo total gasto para processar o GSC completo (segundos): 2.8984832763671875

Arquivo 104 de 105 -> 99.04761904761905%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.8437983989715576

Arquivo 105 de 105 -> 100.0%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.564863443374634

Tempo médio para processar o GSC em cada arquivo com SNR -20: 1.7086626393454416

INICIANDO A SIMULAÇÃO COM SNR -19

Arquivo 1 de 105 -> 0.9523809523809524%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.8179996013641357

Arquivo 2 de 105 -> 1.9047619047619049%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.6374831199645996

Arquivo 3 de 105 -> 2.857142857142857%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.084885597229004

Arquivo 4 d

Tempo total gasto para processar o GSC completo (segundos): 0.590397834777832

Arquivo 50 de 105 -> 47.61904761904761%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 4.7881858348846436

Arquivo 51 de 105 -> 48.57142857142857%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.4323272705078125

Arquivo 52 de 105 -> 49.523809523809526%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.51912522315979

Arquivo 53 de 105 -> 50.476190476190474%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.654176950454712

Arquivo 54 de 105 -> 51.42857142857142%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.2234292030334473

Arquivo 55 de 105 -> 52.38095238095239%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o G

Tempo total gasto para processar o GSC completo (segundos): 0.7862296104431152

Arquivo 102 de 105 -> 97.14285714285714%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.198486566543579

Arquivo 103 de 105 -> 98.09523809523809%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 3.0420665740966797

Arquivo 104 de 105 -> 99.04761904761905%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.8607871532440186

Arquivo 105 de 105 -> 100.0%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.170323133468628

Tempo médio para processar o GSC em cada arquivo com SNR -19: 1.5077024028414772

INICIANDO A SIMULAÇÃO COM SNR -18

Arquivo 1 de 105 -> 0.9523809523809524%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.9899656772613525

Arquivo 

...

Tempo total gasto para processar o GSC completo (segundos): 4.093395948410034

Arquivo 77 de 105 -> 73.33333333333333%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 4.2188920974731445

Arquivo 78 de 105 -> 74.28571428571429%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.399066686630249

Arquivo 79 de 105 -> 75.23809523809524%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.0000100135803223

Arquivo 80 de 105 -> 76.19047619047619%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.2296366691589355

Arquivo 81 de 105 -> 77.14285714285715%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.630298137664795

Arquivo 82 de 105 -> 78.0952380952381%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC

Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.6529173851013184

Arquivo 23 de 105 -> 21.904761904761905%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.408893346786499

Arquivo 24 de 105 -> 22.857142857142858%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.492327928543091

Arquivo 25 de 105 -> 23.809523809523807%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 4.43460488319397

Arquivo 26 de 105 -> 24.761904761904763%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 6.3774871826171875

Arquivo 27 de 105 -> 25.71428571428571%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 3.6878011226654053

Arquivo 28 de 105 -> 26.666666666666668%
Iniciando a filtragem adaptativa 

Tempo total gasto para processar o GSC completo (segundos): 1.959930181503296

Arquivo 74 de 105 -> 70.47619047619048%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.5731101036071777

Arquivo 75 de 105 -> 71.42857142857143%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.8607065677642822

Arquivo 76 de 105 -> 72.38095238095238%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 3.7435805797576904

Arquivo 77 de 105 -> 73.33333333333333%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 4.964967727661133

Arquivo 78 de 105 -> 74.28571428571429%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.2444918155670166

Arquivo 79 de 105 -> 75.23809523809524%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o G

Tempo total gasto para processar o GSC completo (segundos): 1.3301560878753662

Arquivo 20 de 105 -> 19.047619047619047%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.657169818878174

Arquivo 21 de 105 -> 20.0%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 3.319899797439575

Arquivo 22 de 105 -> 20.952380952380953%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.1657941341400146

Arquivo 23 de 105 -> 21.904761904761905%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.185587167739868

Arquivo 24 de 105 -> 22.857142857142858%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.4721999168395996

Arquivo 25 de 105 -> 23.809523809523807%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC comple

Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 3.5754969120025635

Arquivo 72 de 105 -> 68.57142857142857%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.9605329036712646

Arquivo 73 de 105 -> 69.52380952380952%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 2.0466418266296387

Arquivo 74 de 105 -> 70.47619047619048%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 1.1936304569244385

Arquivo 75 de 105 -> 71.42857142857143%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 0.9434316158294678

Arquivo 76 de 105 -> 72.38095238095238%
Iniciando a filtragem adaptativa com LMS
Tempo total gasto para processar o GSC completo (segundos): 4.028949737548828

Arquivo 77 de 105 -> 73.33333333333333%
Iniciando a filtragem adaptativa co